# Eat Safe, Love

## Notebook Set Up

In [2]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our database
print(db.list_collection_names())

[]


In [6]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene':20}

# Use count_documents to display the number of documents in the result
establishments.count_documents(query)
# Display the first document in the results using pprint
pprint(establishments.find_one(query))

None


In [8]:
# Convert the result to a Pandas DataFrame
hygiene = establishments.find(query)

hygiene_df = pd.DataFrame(hygiene)
# Display the number of rows in the DataFrame
print(hygiene_df.shape[0])
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [9]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query_london_rating = {
    'LocalAuthorityName': {'$regex': "London", '$options': 'i'},
    'RatingValue': {'$in': [4, 5]}}

# Use count_documents to display the number of documents in the result
print("Number of documents in result:", db.establishments.count_documents(query_london_rating))

# Display the first document in the results using pprint
pprint(db.establishments.find_one(query_london_rating))

Number of documents in result: 0
None


In [10]:
# Convert the result to a Pandas DataFrame
london = (establishments.find(query_london_rating))
london_df = pd.DataFrame(london)
# Display the number of rows in the DataFrame
pprint(london_df.shape[0])
# Display the first 10 rows of the DataFrame
london_df.head(10)

0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [19]:
# Search for establishments within 0.01 degree on either side of latitude and longitude
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01

# Get the latitude and longitude of "Penang Flavours"
location = establishments.find_one({'BusinessName': 'Penang Flavours'})

if location and 'geocode' in location:
    latitude = location['geocode'].get('latitude')
    longitude = location['geocode'].get('longitude')

    if latitude is not None and longitude is not None:
        # Define the query
        query_3 = {
            'RatingValue': 5,
            'geocode.latitude': {'$lte': latitude + degree_search, '$gte': latitude - degree_search},
            'geocode.longitude': {'$lte': longitude + degree_search, '$gte': longitude - degree_search}
        }

        # Sort by hygiene score (ascending order)
        sort_3 = [('scores.Hygiene', 1)]

        # Count the number of documents
        count_3 = establishments.count_documents(query_3)
        print(f"Number of documents in result: {count_3}")

        # Print the first document in the results
        pprint(establishments.find_one(query_3, sort=sort_3))
    else:
        print("Latitude or Longitude is missing in the document.")
else:
    print("No document found with BusinessName: 'Penang Flavours'.")

No document found with BusinessName: 'Penang Flavours'.


In [22]:
# Convert the result to a Pandas DataFrame
if 'query_3' in globals():
    df_3 = pd.DataFrame(list(establishments.find(query_3).sort(sort_3).limit(5)))

    # Display the number of rows in the DataFrame
    print(f"Rows in this DataFrame: {len(df_3)}")

    # Display the first 10 rows of the DataFrame
    df_3.head(10)
else:
    print("Error: query_3 is not defined. Run the first code block first.")

Error: query_3 is not defined. Run the first code block first.


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [13]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match': {'scores.Hygiene': 0}}
# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id': '$LocalAuthorityName', 
                          'count': {'$sum': 1}
                         }
              }
# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': {'count': -1}}
# Print the number of documents in the result
pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))
print(f"The number of documents in the result: {len(results)}")
# Print the first 10 results
pprint(results[0:10])


The number of documents in the result: 0
[]


In [14]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(f"Rows in this DataFrame: {len(result_df)}")
# Display the first 10 rows of the DataFrame
result_df.head(10)

Rows in this DataFrame: 0


""
